In [1]:
import numpy as np
from sympy import *
init_printing(num_columns=200)

def deg_to_rad(angle: float) -> float:
    return angle * pi / 180

class Link:
    def __init__(self, alpha, a, d, theta) -> None:
        self.alpha = alpha
        self.a = a
        self.d = d
        self.theta = theta

L1 = 0.3
L2 = 0.25

theta_1 = np.deg2rad(0)
theta_4 = np.deg2rad(0)
theta_2 = np.deg2rad(0)
theta_3 = np.deg2rad(0)

links = [
    Link(pi / 2,          0,              0,                theta_1 - pi / 2),
    Link(-pi / 2,         0,              0,                theta_2 + pi / 2),
    Link(pi / 2,          0,             L1,                theta_3 + pi / 2),
    Link(-pi / 2,         0,              0,                theta_4 - pi / 2),
    Link(0,           L2,             0,                0),
]

def create_matrix(links, i):
    mat = Matrix([
            [cos(links[i].theta),                       -sin(links[i].theta),                       0,                      links[i].a],
            [sin(links[i].theta) * cos(links[i].alpha), cos(links[i].theta) * cos(links[i].alpha),  -sin(links[i].alpha),   -sin(links[i].alpha) * links[i].d],
            [sin(links[i].theta) * sin(links[i].alpha), cos(links[i].theta) * sin(links[i].alpha),  cos(links[i].alpha),    cos(links[i].alpha) * links[i].d],
            [0,                                         0,                                          0,                      1]
        ])
    return mat

In [2]:
T01 = create_matrix(links, 0)
T12 = create_matrix(links, 1)
T23 = create_matrix(links, 2)
T34 = create_matrix(links, 3)
T45 = create_matrix(links, 4)

In [3]:
R01 = T01[:3,:3]
R12 = T12[:3,:3]
R23 = T23[:3,:3]
R34 = T34[:3,:3]
R45 = T45[:3,:3]

p1 = T01[:3, 3]
p3 = T23[:3, 3]
p2 = T12[:3, 3]
p4 = T34[:3, 3]
p5 = T45[:3, 3]

In [6]:

def cross(df1, df2):
    x1 = df1[0]
    y1 = df1[1]
    z1 = df1[2]

    x2 = df2[0]
    y2 = df2[1]
    z2 = df2[2]
    return Matrix([[y1*z2 - y2*z1],
                    [z1*x2 - z2*x1],
                    [x1*y2 - x2*y1]])

Fx, Fy, Fz = 10, 8, 5
f5 = Matrix([[Fx], [Fy], [Fz]])
n5 = Matrix([[0], [0], [0]])

f4 = R45 @ f5
n4 = R45 @ n5 + cross(p5, f4)

f3 = R34 @ f4
n3 = R34 @ n4 + cross(p4, f3)

f2 = R23 @ f3
n2 = R23 @ n3 + cross(p3, f2)

f1 = R12 @ f2
n1 = R12 @ n2 + cross(p2, f1)

In [7]:
z = np.array([0, 0, 1])

tau5 = n5.T @ z
tau4 = n4.T @ z
tau3 = n3.T @ z
tau2 = n2.T @ z
tau1 = n1.T @ z

In [8]:
Tau = Matrix([tau1, tau2, tau3, tau4])

In [9]:
Tau

⎡ 4.4 ⎤
⎢     ⎥
⎢-2.75⎥
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎣ 2.0 ⎦

In [11]:
w0 = np.zeros(3)
v0 = np.zeros(3)
dth1, dth2,dth3, dth4 =0,0,0,0

w1 = R01.T @ w0 + np.array([0 ,0, dth1])
v1 = R01.T @ (np.reshape(v0, (3, 1)) + cross(w0, p1))

w2 = R12.T @ w1 + np.array([0 ,0, dth2])
v2 = R12.T @ (np.reshape(v1, (3, 1)) + cross(w1, p2))

w3 = R23.T @ w2 + np.array([0 ,0, dth3])
v3 = R23.T @ (np.reshape(v2, (3, 1)) + cross(w2, p3))

w4 = R34.T @ w3 + np.array([0 ,0, dth4])
v4 = R34.T @ (np.reshape(v3, (3, 1)) + cross(w3, p4))

w5 = R45.T @ w4 + np.array ([0 ,0, 0])
v5 = R45.T @ (np.reshape(v4, (3, 1)) + cross(w4, p5))

In [12]:
vw = ([v4[0], v4[1], v4[2], w4[0], w4[1], w4[2]])
jacob = jacobian(vw, ([dth1, dth2, dth3, dth4]))

NameError: name 'jacobian' is not defined

In [18]:
jac = np.linalg.solve(np.reshape(vw, (6, 1)), np.reshape(([dth1, dth2, dth3, dth4,  0, 0]), (4, 1)))

ValueError: cannot reshape array of size 6 into shape (4,1)